In [1]:
import sqlite3
import pandas as pd

In [50]:
class ChinookDB:
    def __init__(self, dbpath):
        self.dbpath = dbpath
        self.conn = sqlite3.connect(dbpath)
        self.cur = self.conn.cursor()

    def group_by_year(self):
        query = """
            SELECT SUBstr(InvoiceDate, 1, 4) AS Year, SUM(Total) 
            FROM invoices 
            GROUP BY Year
        """
        return pd.read_sql_query(query, self.conn)

    def total_sales_cntr(self):
        query = """
            SELECT Country, SUM(Total) 
            FROM invoices as i 
            JOIN customers  as c 
            ON i.CustomerId=c.CustomerId 
            GROUP BY Country
            """
        return pd.read_sql_query(query, self.conn)

    def total_sales_cntr(self):
        query = """
        SELECT Country, SUM(Total) 
        FROM invoices as i 
        JOIN customers  as c 
        ON i.CustomerId=c.CustomerId 
        GROUP BY Country
        """
        return pd.read_sql_query(query, self.conn)

    def track_count(self):
        query = """
        SELECT PlaylistId, COUNT(TrackId)
        FROM playlist_track
        GROUP BY PlaylistId
        """
        return pd.read_sql_query(query, self.conn)

    def track_length(self):
        query = """
        SELECT PlaylistId, SUM(Milliseconds /60000) as Minutes
        FROM playlist_track as pt
        JOIN tracks as t ON pt.TrackId=t.TrackId
        GROUP BY PlaylistId
        """
        return pd.read_sql_query(query, self.conn)
    
    def track_length_name(self):
        query = """
        SELECT PlaylistId, Name, SUM(Milliseconds /60000) as Minutes 
        FROM playlist_track as pt 
        JOIN tracks as t ON pt.TrackId=t.TrackId
        JOIN playlists as pl ON pl.PlaylistId=pt.PlaylistId
        GROUP BY Name
        """
        return pd.read_sql_query(query, self.conn)

    def total_sales_by_agent(self):
        query = """
        SELECT e.FirstName, e.LastName, SUM(i.Total) as Total_Sales
        FROM invoices as i
        JOIN customers as c ON i.CustomerId=c.CustomerId
        JOIN employees as e ON c.SupportRepId=e.EmployeeId 
        GROUP BY e.EmployeeId
        """
        return pd.read_sql_query(query, self.conn)
    
    def top_selling(self):
        query = """
        SELECT t.Name,
        COUNT(ii.Quantity) AS Quantity,
        SUBstr(i.InvoiceDate, 1, 4) AS Year
        FROM tracks as t
        JOIN invoice_items as ii ON t.TrackId = ii.TrackId
        JOIN invoices as i ON ii.InvoiceId = i.InvoiceId
        GROUP BY t.Name
        HAVING YEAR = '2012'
        ORDER BY Quantity DESC
        """
        return pd.read_sql_query(query, self.conn)

    def __del__(self):
        print("Closing connection")
        self.conn.close()

In [51]:
db = ChinookDB("chinook.db")
db.dbpath

Closing connection


'chinook.db'

In [ ]:
# 1. get total sales for all years using invoice table
# you will want to use SUBSTR to get the year from the invoice date
# you will want to use SUM to get the total sales for each year
# 2. get total sales for each country - use invoice table
# you will also need to join with the customer table - those have country info
#3. a count tracks in each playlist - use playlist_track table
#extra challenge get total track lenght in minutes for each playlist
# you will need to join with the track table
# 3. c cherry on top - provide names of these playlists
# so you will want to join with the playlist table as well
# Task 1
# provide a query that shows total sales by each sales agent
# will require to join with the customer table
# will require to join with the invoice table
# Task 2
# QUERY TO find the top selling track of 2012
# will require use track table
# will requiret to join with invoice items table
# will require to join with the invoice table

In [47]:
db.group_by_year()

,Year,SUM(Total)
0,2009,449.46
1,2010,481.45
2,2011,469.58
3,2012,477.53
4,2013,450.58


In [16]:
db.total_sales_cntr().head()

,Country,SUM(Total)
0,Argentina,37.62
1,Australia,37.62
2,Austria,42.62
3,Belgium,37.62
4,Brazil,190.10


In [19]:
db.total_sales_cntr().head()

,Country,SUM(Total)
0,Argentina,37.62
1,Australia,37.62
2,Austria,42.62
3,Belgium,37.62
4,Brazil,190.10


In [20]:
db.track_count().head()

,PlaylistId,COUNT(TrackId)
0,1,3290
1,3,213
2,5,1477
3,8,3290
4,9,1


In [21]:
db.track_length().head()

,PlaylistId,Minutes
0,1,12981
1,3,8239
2,5,5914
3,8,12981
4,9,4


In [52]:
db.total_sales_by_agent().head()

,FirstName,LastName,Total_Sales
0,Jane,Peacock,833.04
1,Margaret,Park,775.40
2,Steve,Johnson,720.16


In [45]:
db.top_selling().head()

,Name,Quantity,Year
0,Release,3,2012
1,You've Got Another Thing Comin',2,2012
2,Wrathchild,2,2012
3,Waiting,2,2012
4,The Evil That Men Do,2,2012


In [53]:
del db

Closing connection
